# FLIC: Data Analysis Part 1 — Introduction and DFM Data
**Scott Pletcher and the Pletcher Lab**

---

## Overview

This notebook begins a series of notebooks that contain working examples intended to make it easier to understand how we analyze the signal data from FLIC DFMs.  This Python implementation (`pyflic`) is a full port of the original R code, with an object-oriented design centred on the **`Experiment`** class as the primary interface.

We have written functions that cover simple feeding experiments (one feeding well per chamber) and choice experiments (two feeding wells per chamber).  These notebooks currently serve as the primary documentation.  The source code is commented and available from GitHub.  Example data files are included in the repository under `python_test_data/`.

As with all FLIC code this is a work in progress.  If you decide to extend the code and wish to share it with the community, please contact [Scott](mailto:spletch@umich.edu).

## Installation

Install `pyflic` from GitHub using `uv` (recommended):

```bash
uv add git+https://github.com/PletcherLab/pyflic.git
```

if you want to install from a specific tagged commit (say, called 'tagname')

```bash
uv add git+https://github.com/PletcherLab/pyflic.git@tagname
```


Or from a distributed wheel file:

```bash
uv add pyflic-0.1.0-py3-none-any.whl
```

See [INSTALL.md](../../INSTALL.md) for full instructions including pip alternatives and version pinning.

## Command-Line Tools

After installation, three commands are available from the terminal:

| Command | Description |
|---|---|
| `pyflic` | Print a summary of available commands |
| `pyflic-config` | Launch the experiment config editor GUI |
| `pyflic-qc <project_dir>` | Launch the QC viewer for a project directory |

**Config editor** — use this to create or edit `flic_config.yaml` for a new experiment:

```bash
pyflic-config
```

**QC viewer** — launch an interactive viewer for a project directory:

```bash
pyflic-qc /path/to/my_experiment
```

Running `pyflic-config` from inside your project directory will automatically load any existing `flic_config.yaml` found there.

## Getting Started

1. Install [Python 3.13+](https://www.python.org/) for your operating system.
2. Install [JupyterLab](https://jupyter.org/) (or VS Code with the Jupyter extension) to run these notebooks interactively.
3. Create a **project directory** on your hard drive.  Organise it as follows:

   ```
   my_experiment/
     flic_config.yaml   ← experiment configuration (DFMs, treatments, parameters)
     data/              ← DFM CSV files (e.g. DFM_1.csv, DFM_2_1.csv …)
     qc/                ← QC reports are written here
     analysis/          ← summary figures and CSVs are written here
   ```

4. Copy all DFM data files (`DFM_N.csv` or multi-segment `DFM_N_M.csv`) into the `data/` subdirectory.
5. Write (or copy) a `flic_config.yaml` that declares the experimental design. If you do not have one, you can create one using the configuration editor outlined in the code cell below.

All examples in this notebook use the data in `python_test_data/`.

In [ ]:
import sys
from pathlib import Path

# Locate the pyflic repository root (the directory that contains pyproject.toml).
def _find_repo_root(marker: str = "pyproject.toml") -> Path:
    for p in [Path().resolve(), *Path().resolve().parents]:
        if (p / marker).exists():
            return p
    raise RuntimeError(f"Could not find repo root ({marker} not found in any parent).")

repo_root = _find_repo_root()
if str(repo_root.parent) not in sys.path:
    sys.path.insert(0, str(repo_root.parent))

# The three classes you will use most often.
from pyflic import Experiment, DFM, Parameters



## Project Directory   

It is required that you set your project directory. If this notebook is in that directory, specify it as "./". 

In [ ]:
# Point at the example project directory.
project_dir = "./"


## Configuration Editor   

If needed, you can 

## Parameters

To identify food interactions we must specify a set of thresholds and settings.  In `pyflic` these are stored in an immutable `Parameters` object.  Two factory methods mirror the R equivalents:

| R | Python |
|---|---|
| `ParametersClass.TwoWell()` | `Parameters.two_well()` |
| `ParametersClass.SingleWell()` | `Parameters.single_well()` |

In normal use, parameters are set once in `flic_config.yaml` and applied automatically when the experiment is loaded (see §&nbsp;**Loading an Experiment**).  You only need to construct `Parameters` directly when loading individual DFMs outside of a project.

In [ ]:
# Default two-well (choice) parameters.
p_choice = Parameters.two_well()

# Inspect every field and its current value.
import dataclasses
for f in dataclasses.fields(p_choice):
    val = getattr(p_choice, f.name)
    if hasattr(val, 'tolist'):   # numpy array
        val = val.tolist()
    print(f"{f.name:35s} {val}")

| Parameter | Definition |
|---|---|
| `baseline_window_minutes` | Window size (minutes) used to normalise the non-feeding signal to zero via a running median. |
| `feeding_threshold` | Normalised signal above which a feeding interaction is indicated. |
| `feeding_minimum` | Minimum normalised signal to be linked to a threshold-crossing event. |
| `tasting_minimum` | Lower bound of the tasting signal range. |
| `tasting_maximum` | Upper bound of the tasting signal range (must be ≤ `feeding_minimum` to avoid overlap). |
| `feeding_minevents` | Minimum number of consecutive feeding signals required to constitute a feeding event. |
| `tasting_minevents` | Minimum number of consecutive tasting signals required to constitute a tasting event. |
| `samples_per_second` | Signal sample frequency (normally 5 for the standard MCU). Do not modify. |
| `chamber_size` | Number of feeding wells per chamber (1 for single-well, 2 for choice). |
| `chamber_sets` | Encoding of well numbers into chamber IDs. Do not modify. |
| `feeding_event_link_gap` | Contiguous lick bouts are merged if separated by fewer than this many seconds of inactivity. |
| `pi_direction` | Which physical side of the DFM is WellA.  `"left"` means the left well is WellA (PI&nbsp;multiplier&nbsp;=&nbsp;+1 in R); `"right"` swaps this.  A positive PI indicates preference for WellA. |
| `correct_for_dual_feeding` | Whether to subtract signal artefacts caused by simultaneous feeding in adjacent wells. |

The feeding and tasting parameters are critical; defaults match those commonly used in the Pletcher lab.  To avoid biasing tasting data, always ensure `tasting_maximum ≤ feeding_minimum`.

In [ ]:
# Parameters are immutable; use with_updates() to get a modified copy.
# This mirrors R's SetParameter().
p_modified = p_choice.with_updates(
    feeding_threshold=30,
    feeding_minimum=20,
    tasting_minimum=10,
    tasting_maximum=20,
)
print("feeding_threshold :", p_modified.feeding_threshold)
print("feeding_minimum   :", p_modified.feeding_minimum)

# p_choice itself is unchanged.
print("original threshold:", p_choice.feeding_threshold)

In [ ]:
# For single-well experiments.
p_single = Parameters.single_well()
print("chamber_size:", p_single.chamber_size)

## Loading an Experiment

The recommended way to load a complete experiment is `Experiment.load()`.  It reads the `flic_config.yaml` in the project directory, loads all DFM CSV files in parallel, applies the declared parameters, and builds the experimental design (treatment assignments).

```python
exp = Experiment.load(project_dir)
```

The `flic_config.yaml` specifies:
- Global and per-DFM parameters.
- Which chambers on each DFM belong to each treatment group.

Loading is multi-threaded by default (`parallel=True`, `executor="threads"`) and uses as many workers as the machine can efficiently provide.  An optional `range_minutes=(start, end)` tuple limits how much data is loaded from each CSV, which can be useful for large overnight files.

```python
# Load only the first two hours of data.
exp = Experiment.load(project_dir, range_minutes=(0, 120))
```

In [ ]:
exp = Experiment.load(project_dir)
print("DFMs loaded:", sorted(exp.dfms.keys()))

## Exploring the Experiment

`summary_text()` returns a human-readable report covering the configuration, DFM details, experimental design, and a QC summary.

In [ ]:
print(exp.summary_text())

In [ ]:
# The experimental design as a tidy DataFrame (DFM, Chamber, Treatment).
exp.design.design_table().sort_values(["Treatment", "DFM", "Chamber"])

## Feeding and Tasting Interactions

When a DFM is loaded, the following calculations are performed automatically for each DFM:

1. **Load raw data** from the relevant CSV file(s).  Multi-segment experiments (files named `DFM_N_0.csv`, `DFM_N_1.csv`, …) are stitched together automatically.

2. **Calculate elapsed time** using date/time columns to produce a `Minutes` column for each observation.

3. **Baseline normalisation.** A running median is applied to each well's signal with a window defined by `baseline_window_minutes` (default 3 min, i.e. 900 samples at 5 Hz).  The assumption is that feeding interactions in any 3-minute window will be sufficiently rare that the median represents background signal.  The baselined signal is the raw signal minus this running-median baseline.  This corrects for inter-DFM signal variation and for slow drifts (e.g. from over-filled food wells).

4. **Identify feeding licks and events.**  An observation is classified as a feeding lick if:
   - it exceeds `feeding_threshold`, **or**
   - it exceeds `feeding_minimum` and is adjacent to at least one sample that exceeds `feeding_threshold`.
   
   A contiguous run of licks forms a candidate *event*.  The event is retained only if the number of licks meets or exceeds `feeding_minevents`.  Adjacent events separated by less than `feeding_event_link_gap` seconds of inactivity are merged into a single event.

5. **WellA / WellB assignment (two-well experiments).**  The `pi_direction` parameter maps physical well positions to the logical WellA/WellB labels used in all output.  `pi_direction="left"` makes the left well WellA (PI multiplier = +1 in the R code); `"right"` reverses this.  Swapping `pi_direction` across DFMs while counterbalancing food positions ensures that a positive PI always indicates preference for WellA regardless of its physical location.

6. **Identify tasting licks.**  Signals that fall between `tasting_minimum` and `tasting_maximum` and were *not* already classified as feeding licks are labelled tasting licks.  Contiguous runs are grouped into tasting events subject to `tasting_minevents`.

7. **Compute PI (two-well experiments).**  At each sample a PI of +1, −1, or 0 is recorded depending on whether WellA, WellB, or neither registered a feeding lick.

8. **Compute durations and intervals.**  The duration (seconds) of each feeding/tasting event and the inter-event interval are stored for downstream analysis.

9. **Lights.**  If the DFM CSV contains `OptoCol1` (and optionally `OptoCol2`) columns, the per-well light state is decoded from the bit-encoded values and stored in `lights_df`.  This is used to place light-on markers on cumulative lick plots.

All computed data is stored on the `DFM` object and accessed through the `Experiment` API described below.

## Quality Control

Before examining results it is important to check data quality.  `compute_qc_results()` inspects each DFM for:

- **Data integrity** — row count, elapsed time consistency, monotone sample index.
- **Data breaks** — gaps in the `Seconds` column larger than `multiplier × median_interval`.  Breaks most often arise from noisy communication (e.g., DFMs in an incubator) and appear as spurious periods of zero feeding.  A return of `None` or count of 0 indicates no problems.
- **Simultaneous feeding** (two-well only) — how often both wells in a chamber register feeding at the same sample, which can indicate signal bleed-through.
- **Bleeding** (two-well only) — whether the signal from one well appears in the adjacent well above a normalised cutoff.

`write_qc_reports()` runs the same checks and writes per-DFM CSV/TXT files plus raw data plots to the `qc/` subdirectory.

In [ ]:
# Inspect QC results in-memory (no files written).
qc = exp.compute_qc_results()

for dfm_id, r in qc.items():
    n_breaks = r["data_breaks_count"]
    integrity = r["integrity"]
    print(f"DFM {dfm_id}: {integrity['n_rawdata']} rows, "
          f"~{integrity['elapsed_minutes_from_minutes_col']:.1f} min, "
          f"{n_breaks} data break(s)")

In [ ]:
# Write full QC reports (CSV, TXT, and a raw-data PNG) to project_dir/qc/.
# Comment this out if you do not want to write files during an exploratory session.
# out_dir = exp.write_qc_reports()
# print("QC reports written to:", out_dir)

## Visualising Raw and Baselined Signals

Raw and baselined signal plots for a single DFM are available through `exp.get_dfm()`.  These plots can reveal baseline drift, abnormal wells, or artefacts before any threshold-based analysis.

Dramatic or persistent changes in the raw data trace are cause for concern.  Note how a slightly elevated baseline in one well is corrected in the normalised (baselined) trace.

Both functions accept an optional `range_minutes=(start, end)` argument to zoom in on a particular period.

In [ ]:
dfm1 = exp.get_dfm(1)

# Full-experiment raw signal plot for DFM 1.
fig = dfm1.plot_raw()
fig.show()

In [ ]:
# Baselined (normalised) signal for the first 100 minutes.
# Overlay the feeding thresholds with include_thresholds=True.
fig = dfm1.plot_baselined(range_minutes=(0, 100), include_thresholds=True)
fig.show()

In [ ]:
# Zoom in on a single well if you need more detail.
fig = dfm1.plot_raw_well(1, range_minutes=(0, 60))
fig.show()

### Direct Data Access

The underlying DataFrames are accessible directly when needed for custom analysis.

| R function | Python equivalent |
|---|---|
| `RawData(dfm)` | `dfm.raw_df` |
| `BaselineData(dfm)` | `dfm.baseline_df` |
| `FirstSampleData(dfm)` | `dfm.raw_df.iloc[0]` |
| `LastSampleData(dfm)` | `dfm.raw_df.iloc[-1]` |

In [ ]:
print("First sample:")
print(dfm1.raw_df.iloc[0])

print("\nLast sample:")
print(dfm1.raw_df.iloc[-1])

In [ ]:
# Head of raw data.
dfm1.raw_df.head()

In [ ]:
# Head of baselined data.
dfm1.baseline_df.head()

## Feeding Summary

`exp.feeding_summary()` aggregates feeding metrics across all DFMs and chambers that have been assigned to a treatment group in the experimental design.  This is the primary data table for statistical analysis.

The optional `range_minutes=(start, end)` argument restricts the analysis to a time window.  `(0, 0)` (the default) uses the entire experiment.

By default licks are transformed to the 0.25 power (`transform_licks=True`) because raw lick counts are right-skewed.  Disable the transformation with `transform_licks=False` when you want raw counts for inspection.

| R function | Python equivalent |
|---|---|
| `Feeding.Summary.DFM(dfm)` | `dfm.feeding_summary()` (single DFM) |
| `Feeding.Summary.DFM(dfm, range=c(0,120))` | `dfm.feeding_summary(range_minutes=(0,120))` |
| *(experiment-level)* | `exp.feeding_summary()` |


In [ ]:
# Experiment-level feeding summary — all treatment-assigned chambers.
fs = exp.feeding_summary()
fs.head(12)

In [ ]:
# Same but with raw (untransformed) lick counts and restricted to the first two hours.
fs_raw = exp.feeding_summary(range_minutes=(0, 120), transform_licks=False)
fs_raw.head(6)

### Column Definitions (two-well / choice experiment)

| Column | Definition |
|---|---|
| `Treatment` | Treatment name as declared in the config. |
| `DFM` | DFM ID number. |
| `Chamber` | Chamber number (1–6 for two-well). |
| `PI` | Preference index calculated from licks over the range. Positive = preference for WellA. |
| `EventPI` | PI calculated from events instead of individual licks. |
| `LicksA` / `LicksB` | Number of feeding licks in WellA / WellB. |
| `EventsA` / `EventsB` | Number of feeding events in WellA / WellB. |
| `MeanDurationA/B` | Mean feeding event duration (seconds) in WellA / WellB. |
| `MedDurationA/B` | Median feeding event duration (seconds) in WellA / WellB. |
| `MeanTimeBtwA/B` | Mean inter-event interval (seconds) in WellA / WellB. |
| `MedTimeBtwA/B` | Median inter-event interval (seconds) in WellA / WellB. |
| `MeanIntA/B` | Mean normalised signal intensity across feeding licks in WellA / WellB. |
| `MedianIntA/B` | Median normalised signal intensity in WellA / WellB. |
| `MinIntA/B` / `MaxIntA/B` | Minimum / maximum signal intensity in WellA / WellB. |
| `StartMin` / `EndMin` | Start and end of the analysis window (minutes). |

For **single-well** experiments the `A/B` suffixes are absent and `PI` / `EventPI` are omitted.

In [ ]:
# You can also get the feeding summary for a single DFM.
dfm1.feeding_summary().head()

### Binned Feeding Summary

`binned_feeding_summary()` divides the time range into non-overlapping bins of a fixed size and computes the feeding summary within each bin.  This lets you track how feeding changes over time.  The same columns as above are produced, plus a `Bin` column identifying each time window.

| R function | Python equivalent |
|---|---|
| `BinnedFeeding.Summary.DFM(dfm, binsize.min=30)` | `dfm.binned_feeding_summary(binsize_min=30)` |

In [ ]:
# 30-minute bins across the full experiment.
bfs = dfm1.binned_feeding_summary(binsize_min=30)
bfs.head(12)

In [ ]:
# 10-minute bins over the first two hours.
dfm1.binned_feeding_summary(binsize_min=10, range_minutes=(0, 120), transform_licks=False).head()

## Cumulative Lick Plots

Cumulative lick plots show how feeding accumulates over time.  They are useful for detecting:
- Whether feeding is uniformly distributed or clustered.
- Whether a preference emerges or reverses during the experiment.
- The timing of any optogenetic stimulation (gold diamond markers appear at every sample where the light is on).

The preferred access point is `exp.plot_cumulative_licks_chamber()`, which automatically adds the treatment name to the plot title.

| R function | Python equivalent |
|---|---|
| `CumulativeLicksPlots.DFM(dfm)` | `dfm.plot_cumulative_licks()` |
| — | `exp.plot_cumulative_licks_chamber(dfm_id, chamber)` |

In [ ]:
# Cumulative licks for DFM 1, Chamber 1.  Treatment label is added automatically.
# single_plot=True (default) overlays WellA and WellB on the same axes.
fig = exp.plot_cumulative_licks_chamber(dfm_id=1, chamber=1)
fig.show()

In [ ]:
# Use single_plot=False to display WellA and WellB in separate stacked panels.
fig = exp.plot_cumulative_licks_chamber(dfm_id=1, chamber=1, single_plot=False)
fig.show()

In [ ]:
# Full DFM cumulative lick plot — one line per well.
# For choice experiments this shows all 12 wells coloured by well number.
fig = dfm1.plot_cumulative_licks()
fig.show()

## Binned Lick Plots

`plot_binned_licks()` shows feeding in fixed-size time bins rather than cumulatively.  For choice experiments a stacked bar chart displays WellA and WellB contributions per chamber in each bin.

Disabling the lick transformation (`transform_licks=False`) can be more intuitive during QC because the y-axis shows raw lick counts.

| R function | Python equivalent |
|---|---|
| `BinnedLicksPlot.DFM(dfm, binsize.min=20)` | `dfm.plot_binned_licks(binsize_min=20)` |

In [ ]:
fig = dfm1.plot_binned_licks(binsize_min=30)
fig.show()

In [ ]:
# Raw (untransformed) lick counts — often clearest for QC.
fig = dfm1.plot_binned_licks(binsize_min=30, transform_licks=False)
fig.show()

## Intervals and Durations

`interval_data()` and `duration_data()` return per-event DataFrames with the timing of each feeding bout.  These are the raw records from which the summary statistics (MeanDuration, MedTimeBtw, …) are computed.

| R function | Python equivalent |
|---|---|
| `GetIntervalData.DFM(dfm)` | `dfm.interval_data()` |

In [ ]:
idata = dfm1.interval_data()
idata.head(10)

In [ ]:
ddata = dfm1.duration_data()
ddata.head(10)

### Per-Chamber / Per-Well Data Access

For two-well chambers you must specify `well='a'` or `well='b'`; for single-well chambers the `well` argument is omitted.

| Accessor | Returns |
|---|---|
| `dfm.licks_for(chamber, 'a')` | Boolean Series — True at each feeding lick sample. |
| `dfm.events_for(chamber, 'a')` | Integer Series — event count at each sample. |
| `dfm.baseline_for(chamber, 'a')` | Float Series — baselined signal. |
| `dfm.durations_for(chamber, 'a')` | DataFrame of bout durations (seconds). |
| `dfm.intervals_for(chamber, 'a')` | DataFrame of inter-bout intervals (seconds). |
| `dfm.lights_for(chamber, 'a')` | Boolean Series — True when the light is on. |

In [ ]:
# Boolean lick Series for Chamber 1, WellA of DFM 1.
licks_a = dfm1.licks_for(1, 'a')
print("Total WellA licks (samples):", licks_a.sum())

licks_b = dfm1.licks_for(1, 'b')
print("Total WellB licks (samples):", licks_b.sum())

In [ ]:
# Bout durations for Chamber 1, WellA.
dfm1.durations_for(1, 'a').head(10)

## Summary Plots

`exp.plot_feeding_summary()` produces a grid of box-and-dot plots, one panel per feeding metric, grouped by treatment.  Individual chamber values are shown as jittered dots overlaid on a box plot.

For two-well experiments this includes PI, EventPI, and per-well (A/B) versions of all metrics.

In [ ]:
fig = exp.plot_feeding_summary()
fig.show()

In [ ]:
# Same plot restricted to the first two hours.
fig = exp.plot_feeding_summary(range_minutes=(0, 120))
fig.show()

## Publication-Quality Well Comparison (plotnine)

`exp.facet_plot_well_durations()` produces a plotnine (ggplot-style) jitter plot comparing WellA and WellB feeding duration across all treatment groups.  Each point is one chamber.  The cross and error bars show the mean ± SEM.

You can provide custom x-axis labels to replace the generic "WellA" / "WellB" with the actual food names used in your experiment — for example `{"WellA": "Sucrose", "WellB": "Yeast"}`.

The `metric` argument selects which duration column to plot (default `"MedDuration"`; use `"MeanDuration"` for the mean).

> **Requires:** `pip install plotnine`

In [ ]:
p = exp.facet_plot_well_durations(
    metric="MedDuration",
    title="Median Feeding Duration by Well and Treatment",
    y_label="Median Duration (s)",
)
p

In [ ]:
# Supply descriptive food names for the x-axis.
# Keys are case-insensitive: "wella" and "WellA" both work.
p = exp.facet_plot_well_durations(
    metric="MedDuration",
    title="Median Feeding Duration by Food and Treatment",
    y_label="Median Duration (s)",
    x_labels={"WellA": "Sucrose", "WellB": "Yeast"},
)
p

In [ ]:
# plot_jitter_summary() is the lower-level function used by facet_plot_well_durations().
# You can call it directly with any feeding summary DataFrame and any pair of columns.
fs = exp.feeding_summary()
p = exp.plot_jitter_summary(
    fs,
    x_col="Treatment",
    y_col="PI",
    facet_col="DFM",
    title="PI by Treatment, faceted by DFM",
    y_label="Preference Index",
)
p

## Running the Complete Analysis

`execute_basic_analysis()` is a convenience wrapper that runs all four standard output steps in sequence with progress messages:

1. `write_qc_reports()` — per-DFM integrity, data-breaks, simultaneous-feeding, bleeding checks, and raw-data plots → `project_dir/qc/`
2. `write_summary()` — human-readable experiment summary → `project_dir/analysis/summary.txt`
3. `write_feeding_summary()` — feeding summary CSV → `project_dir/analysis/feeding_summary.csv`
4. `write_feeding_summary_plot()` — feeding summary figure → `project_dir/analysis/feeding_summary.png`

In [ ]:
# Uncomment to run the full analysis and write all output files.
# exp.execute_basic_analysis()

In [ ]:
# You can also call each step individually.
# exp.write_qc_reports()                # → project_dir/qc/
# exp.write_summary()                   # → project_dir/analysis/summary.txt
# exp.write_feeding_summary()           # → project_dir/analysis/feeding_summary.csv
# exp.write_feeding_summary_plot()      # → project_dir/analysis/feeding_summary.png

---

## Advanced: Loading Individual DFMs Without a Config

For quick exploratory work or when a YAML config is not yet set up, you can load a single DFM directly using `DFM.load()`.  You must construct a `Parameters` object manually.

| R function | Python equivalent |
|---|---|
| `DFMClass(3, p.single)` | `DFM.load(3, p_single, data_dir=...)` |
| `DFMClass(3, p.single, range=c(0,120))` | `DFM.load(3, p_single, data_dir=..., range_minutes=(0,120))` |

In [ ]:
data_dir = project_dir / "data"

# Load DFM 1 with two-well (choice) parameters.
dfm_choice = DFM.load(1, Parameters.two_well(), data_dir=data_dir)
print(f"DFM {dfm_choice.id}: {len(dfm_choice.raw_df)} rows, "
      f"{len(dfm_choice.chambers)} chambers")

In [ ]:
# Load only the first 120 minutes — useful for large overnight files.
dfm_focused = DFM.load(1, Parameters.two_well(), data_dir=data_dir, range_minutes=(0, 120))
print(f"Rows in focused load: {len(dfm_focused.raw_df)}")

### Changing Parameters After Loading

`dfm.with_params()` returns a new `DFM` object with all computations redone under the new parameters.  The original object is unchanged.  This mirrors R's `ChangeParameterObject()`.

| R | Python |
|---|---|
| `ChangeParameterObject(DFM3, p.choice)` | `dfm3.with_params(p_choice)` |

In [ ]:
# Raise the feeding threshold and recompute.
p_strict = Parameters.two_well().with_updates(feeding_threshold=30, feeding_minimum=20)
dfm_strict = dfm_choice.with_params(p_strict)

orig_licks  = dfm_choice.feeding_summary()["LicksA"].sum()
strict_licks = dfm_strict.feeding_summary()["LicksA"].sum()

print(f"WellA lick total — default threshold: {orig_licks:.2f}")
print(f"WellA lick total — strict  threshold: {strict_licks:.2f}")

### Checking Data Breaks on a Single DFM

`dfm.data_breaks()` returns a DataFrame of gap locations, or `None` if no breaks were detected.  A return of `None` (or an empty DataFrame) is the expected result for a clean recording.

| R | Python |
|---|---|
| `FindDataBreaks(DFM11)` | `dfm.data_breaks()` |

In [ ]:
breaks = dfm_choice.data_breaks()
if breaks is None or breaks.empty:
    print("No data breaks detected.")
else:
    print(f"{len(breaks)} break(s) found:")
    print(breaks.head())

---

## Function Reference

Listed below are the principal functions used in this notebook with their signatures.  You should understand what each does and what each argument represents before moving on to the grouped-analysis and choice-chamber documentation.

### `Experiment` (primary API)

```python
Experiment.load(
    project_dir,
    *,
    range_minutes=(0, 0),
    parallel=True,
    max_workers=None,
    executor="threads",
)

exp.get_dfm(dfm_id)                          # → DFM

exp.summary_text(
    *, include_qc=True,
    qc_data_breaks_multiplier=4.0,
    qc_bleeding_cutoff=50.0,
)                                            # → str

exp.compute_qc_results(
    *, data_breaks_multiplier=4.0,
    bleeding_cutoff=50.0,
)                                            # → dict

exp.write_qc_reports(
    out_dir=None,
    *, data_breaks_multiplier=4.0,
    bleeding_cutoff=50.0,
)                                            # → Path

exp.feeding_summary(
    *, range_minutes=(0, 0),
    transform_licks=True,
)                                            # → pd.DataFrame

exp.write_feeding_summary(path=None, ...)    # → Path

exp.plot_feeding_summary(
    *, range_minutes=(0, 0),
    transform_licks=True,
    ncols=2, figsize=None,
)                                            # → Figure

exp.write_feeding_summary_plot(path=None, *, format="png", ...)

exp.plot_cumulative_licks_chamber(
    dfm_id, chamber,
    *, range_minutes=(0, 0),
    transform_licks=True,
    single_plot=True,
)                                            # → Figure

exp.facet_plot_well_durations(
    *, metric="MedDuration",
    range_minutes=(0, 0),
    transform_licks=True,
    title="",
    y_label=None,
    ylim=None,
    x_labels=None,          # e.g. {"WellA": "Sucrose", "WellB": "Yeast"}
    annotation=None,
    jitter_width=0.25,
    point_size=3.0,
    base_font_size=20.0,
)                                            # → plotnine.ggplot

exp.plot_jitter_summary(
    df, *, x_col, y_col,
    facet_col="Treatment",
    title="",
    x_label=None, y_label=None,
    ylim=None,
    x_order=None,
    x_labels=None,
    colors=None,
    annotation=None,
    jitter_width=0.25,
    point_size=3.0,
    base_font_size=20.0,
)                                            # → plotnine.ggplot

exp.write_summary(path=None)                 # → Path

exp.execute_basic_analysis(
    *, data_breaks_multiplier=4.0,
    bleeding_cutoff=50.0,
    range_minutes=(0, 0),
    transform_licks=True,
    plot_format="png",
    dpi=150,
)

exp.design.design_table()                    # → pd.DataFrame
exp.design.feeding_summary(...)              # → pd.DataFrame  (same as exp.feeding_summary)
exp.design.treatment_for(dfm_id, chamber)   # → str | None
```

### `DFM` (lower-level / advanced)

```python
DFM.load(
    dfm_id, params,
    *, data_dir=".",
    range_minutes=(0, 0),
)

dfm.with_params(new_params)                  # → DFM (recomputed)

dfm.raw_df                                   # pd.DataFrame
dfm.baseline_df                              # pd.DataFrame
dfm.lights_df                                # pd.DataFrame

dfm.plot_raw(*, range_minutes=(0, 0))
dfm.plot_baselined(*, range_minutes=(0, 0), include_thresholds=False)
dfm.plot_raw_well(well, *, range_minutes=(0, 0))
dfm.plot_baselined_well(well, *, range_minutes=(0, 0), include_thresholds=False)
dfm.plot_binned_licks(*, binsize_min=30, range_minutes=(0, 0), transform_licks=True)
dfm.plot_cumulative_licks(*, single_plot=False, transform_licks=True)
dfm.plot_cumulative_licks_chamber(
    chamber, *, range_minutes=(0, 0),
    transform_licks=True,
    single_plot=True,
    treatment=None,
)

dfm.feeding_summary(*, range_minutes=(0, 0), transform_licks=True)
dfm.binned_feeding_summary(*, binsize_min=30, range_minutes=(0, 0), transform_licks=True)
dfm.interval_data(*, range_minutes=(0, 0))
dfm.duration_data(*, range_minutes=(0, 0))

dfm.licks_for(chamber, well=None)           # well='a'/'b' required for two-well
dfm.events_for(chamber, well=None)
dfm.baseline_for(chamber, well=None)
dfm.durations_for(chamber, well=None)
dfm.intervals_for(chamber, well=None)
dfm.lights_for(chamber, well=None)

dfm.data_breaks(*, multiplier=4.0)          # → pd.DataFrame | None
dfm.integrity_report()                      # → dict
```

### `Parameters`

```python
Parameters.two_well()                        # default two-well / choice parameters
Parameters.single_well()                     # default single-well parameters
params.with_updates(**kwargs)               # → new Parameters (immutable)
```